## Data Analysis and Cleaning

In [2]:
import numpy as np
import pandas as pd

In [10]:
all_titles = pd.read_csv('data/title.basics.tsv', sep='\t')
all_ratings = pd.read_csv('data/title.ratings.tsv', sep='\t')

# Filter titles to only include movies and TV series
movies_and_series = all_titles[all_titles['titleType'].isin(['movie', 'tvSeries'])]

# Convert columns to correct data types
all_ratings['averageRating'] = pd.to_numeric(all_ratings['averageRating'], errors='coerce')
all_ratings['numVotes'] = pd.to_numeric(all_ratings['numVotes'], errors='coerce')

# Merge titles and ratings, keeping only movies and TV series
merged_data = movies_and_series.merge(all_ratings, on='tconst', how='inner')

# Filter movies/series with at least 1000 votes
df_filtered = merged_data[merged_data['numVotes'] >= 1000]

# Sort by rating (descending), then by numVotes (descending for tie-breaking)
df_sorted = df_filtered.sort_values(by=['averageRating', 'numVotes'], ascending=[False, False])

# Save to new TSV file
df_sorted.to_csv('data/filtered_sorted_with_ratings.tsv', sep='\t', index=False)

In [11]:
# IMDb-style weighted rating system for top 10,000 movies and series

# Calculate overall statistics for the weighted rating formula
overall_mean_rating = df_filtered['averageRating'].mean()
min_votes_required = df_filtered['numVotes'].quantile(0.75)  # Use 75th percentile as minimum

print(f"Overall mean rating: {overall_mean_rating:.2f}")
print(f"Minimum votes threshold (75th percentile): {min_votes_required:.0f}")

# Apply Bayesian weighted rating formula
# Weighted Rating = (v / (v + m)) * R + (m / (v + m)) * C
# Where: v = votes, m = min votes, R = average rating, C = overall mean
def calculate_weighted_rating(row):
    v = row['numVotes']
    R = row['averageRating']
    m = min_votes_required
    C = overall_mean_rating
    
    weighted_rating = (v / (v + m)) * R + (m / (v + m)) * C
    return weighted_rating

# Add weighted rating column
df_filtered['weightedRating'] = df_filtered.apply(calculate_weighted_rating, axis=1)

# Sort by weighted rating (descending), then by numVotes (descending for tie-breaking)
df_weighted_sorted = df_filtered.sort_values(by=['weightedRating', 'numVotes'], ascending=[False, False])

# Get top 10,000 based on weighted ratings
top_10000_weighted = df_weighted_sorted.head(10000)

# Save the weighted top 10,000 to file
top_10000_weighted.to_csv('data/top_10000_weighted_ratings.tsv', sep='\t', index=False)

print(f"\nTop 10 movies/series by weighted rating:")
print(top_10000_weighted[['primaryTitle', 'averageRating', 'numVotes', 'weightedRating']].head(10))

Overall mean rating: 6.39
Minimum votes threshold (75th percentile): 10202

Top 10 movies/series by weighted rating:
                      primaryTitle  averageRating  numVotes  weightedRating
175388                Breaking Bad            9.5   2405005        9.486874
67193     The Shawshank Redemption            9.3   3104334        9.290476
153336  Avatar: The Last Airbender            9.3    417720        9.230683
129412                    The Wire            9.3    413512        9.229994
176712             Game of Thrones            9.2   2485498        9.188523
39197                The Godfather            9.2   2163561        9.186824
80066                 The Sopranos            9.2    545115        9.148422
162488             The Dark Knight            9.1   3079402        9.091060
308327             Attack on Titan            9.1    649760        9.058146
234194                   Aspirants            9.1    316623        9.015484


C:\Users\franc\AppData\Local\Temp\ipykernel_10268\175845366.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['weightedRating'] = df_filtered.apply(calculate_weighted_rating, axis=1)


## Adding Cast to data we already have

### Clean title.principals.tsv to only have movies/series in top 10000

In [33]:
# clean title.principals.tsv to only have movies/series in top 10000
all_principals = pd.read_csv('data/title.principals.tsv', sep='\t')
top_10000_df = pd.read_csv('data/top_10000_weighted_ratings.tsv', sep='\t')

# Filter all_principals to only include titles in top_10000_df
filtered_principals = all_principals[all_principals['tconst'].isin(top_10000_df['tconst'])]
filtered_principals.to_csv('data/filtered_title_principals_top_10000.tsv', sep='\t', index=False)

### Getting top 3 actor/actress names and their respective characters

In [4]:
principals = pd.read_csv('data/cast_data/filtered_title_principals_top_10000.tsv', sep='\t')
names = pd.read_csv('data/cast_data/name.basics.tsv', sep='\t')
top10000 = pd.read_csv('data/top_10000_weighted_ratings.tsv', sep='\t')

# Sample of top 10 entries for testing - change to top10000 for full processing
top10_sample = top10000.head(10)  # Change to top10000 for full dataset

# Get each tconst from dataset and find the top 3 UNIQUE actors/actresses nconst in principals and their respective characters
def get_top_3_cast(tconst):
    cast = principals[principals['tconst'] == tconst]
    cast = cast[cast['category'].isin(['actor', 'actress'])]
    
    result = []
    seen_nconst = set()  # Track unique nconst values
    
    for _, row in cast.iterrows():
        nconst = row['nconst']
        
        # Skip if we've already seen this nconst
        if nconst in seen_nconst:
            continue
            
        character = row['characters']
        
        # Clean the character field - remove brackets and extra quotes
        if pd.notna(character) and character != '\\N':
            import json
            try:
                # Try to parse as JSON first
                character_list = json.loads(character)
                if isinstance(character_list, list) and len(character_list) > 0:
                    clean_character = character_list[0]  # Get first character name
                else:
                    clean_character = str(character)
            except (json.JSONDecodeError, ValueError):
                # If JSON parsing fails, try manual cleaning
                clean_character = character.strip('[]"').replace('""', '"')
        else:
            clean_character = "Unknown Character"
        
        name_row = names[names['nconst'] == nconst]
        
        if not name_row.empty:
            actor_name = name_row.iloc[0]['primaryName']
            result.append((actor_name, clean_character))
            seen_nconst.add(nconst)
            
            # Stop when we have 3 unique cast members
            if len(result) >= 3:
                break
    
    return result

# Chunk-based processing with automatic saving
from tqdm import tqdm
import os

# Configuration
chunk_size = 500
dataset = top10000  # Change to top10000 for full processing
output_dir = 'data/cast_data/chunks'
final_output = 'data/cast_data/top_10_with_cast.tsv'  # Change to top_10000_with_cast.tsv for full processing

# Create chunks directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

print(f"Processing {len(dataset)} entries in chunks of {chunk_size}...")

# Process in chunks
all_results = []
for chunk_start in tqdm(range(0, len(dataset), chunk_size), desc="Processing chunks"):
    chunk_end = min(chunk_start + chunk_size, len(dataset))
    chunk_data = dataset.iloc[chunk_start:chunk_end].copy()
    
    print(f"\nProcessing chunk {chunk_start//chunk_size + 1}: entries {chunk_start+1} to {chunk_end}")
    
    # Process each entry in the chunk
    chunk_results = []
    for idx, (_, row) in enumerate(tqdm(chunk_data.iterrows(), total=len(chunk_data), desc=f"Chunk {chunk_start//chunk_size + 1}")):
        try:
            cast_result = get_top_3_cast(row['tconst'])
            chunk_results.append(cast_result)
            
            # Print progress every 50 entries within chunk
            if (idx + 1) % 50 == 0:
                print(f"  Processed {idx + 1}/{len(chunk_data)} in current chunk")
                
        except Exception as e:
            print(f"  Error processing {row['tconst']}: {e}")
            chunk_results.append([])  # Empty result for failed entries
    
    # Add results to chunk data
    chunk_data['top_3_cast'] = chunk_results
    
    # Save chunk to file
    chunk_filename = f"{output_dir}/chunk_{chunk_start//chunk_size + 1:03d}_{chunk_start+1:05d}_to_{chunk_end:05d}.tsv"
    chunk_data.to_csv(chunk_filename, sep='\t', index=False)
    print(f"  Saved chunk to: {chunk_filename}")
    
    # Add to all results
    all_results.append(chunk_data)
    
    print(f"  Chunk {chunk_start//chunk_size + 1} completed successfully!")

# Combine all chunks
print("\nCombining all chunks...")
final_dataset = pd.concat(all_results, ignore_index=True)

# Save final combined result
final_dataset.to_csv(final_output, sep='\t', index=False)
print(f"\nProcessing completed! Final results saved to '{final_output}'")
print(f"Total entries processed: {len(final_dataset)}")

# Display sample results
print("\nSample results:")
print(final_dataset[['primaryTitle', 'top_3_cast']].head(10))

# Clean up chunk files (optional - comment out if you want to keep them)
print(f"\nChunk files saved in '{output_dir}' for backup")
print("You can delete the chunks folder once you've verified the final file is correct")

Processing 10000 entries in chunks of 500...


Processing chunks:   0%|          | 0/20 [00:00<?, ?it/s]


Processing chunk 1: entries 1 to 500


Chunk 1:   0%|          | 0/500 [00:00<?, ?it/s]

  Processed 50/500 in current chunk


...

  Processed 400/500 in current chunk


Chunk 20:  80%|████████  | 401/500 [10:25<02:47,  1.69s/it]

  Processed 450/500 in current chunk


Processing chunks: 100%|██████████| 20/20 [4:26:43<00:00, 800.19s/it]

  Processed 500/500 in current chunk
  Saved chunk to: data/cast_data/chunks/chunk_020_09501_to_10000.tsv
  Chunk 20 completed successfully!

Combining all chunks...

Processing completed! Final results saved to 'data/cast_data/top_10_with_cast.tsv'
Total entries processed: 10000

Sample results:
                 primaryTitle  \
0                Breaking Bad   
1    The Shawshank Redemption   
2  Avatar: The Last Airbender   
3                    The Wire   
4             Game of Thrones   
5               The Godfather   
6                The Sopranos   
7             The Dark Knight   
8             Attack on Titan   
9                   Aspirants   

                                          top_3_cast  
0  [(Bryan Cranston, Walter White), (Aaron Paul, ...  
1  [(Tim Robbins, Andy Dufresne), (Morgan Freeman...  
2  [(Dee Bradley Baker, Appa), (Zach Tyler Eisen,...  
3  [(Dominic West, Detective James 'Jimmy' McNult...  
4  [(Emilia Clarke, Daenerys Targaryen), (Peter D...  
5  [(Mar

## Web Scraping for Movie/Series Description

In [32]:
# Web scraping script for StreamWithVPN data
%pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup
import time
import re
from urllib.parse import quote
import pandas as pd
from tqdm import tqdm

# Load the top 10,000 weighted ratings data
top_10000_df = pd.read_csv('data/top_10000_weighted_ratings.tsv', sep='\t')

def clean_title_for_url(title):
    """
    Clean and format movie/series title for URL generation
    """
    # Remove special characters and replace spaces with hyphens
    cleaned = re.sub(r'[^\w\s-]', '', title)
    cleaned = re.sub(r'\s+', '-', cleaned.strip())
    return cleaned.lower()

def generate_streamwithvpn_url(title, year):
    """
    Generate StreamWithVPN URL based on title and year
    Example: "The Wolf's Call" (2019) -> "https://www.streamwithvpn.com/the-wolfs-call-2019"
    """
    clean_title = clean_title_for_url(title)
    # Handle cases where year might be NaN or missing
    if pd.isna(year):
        return f"https://www.streamwithvpn.com/{clean_title}"
    else:
        return f"https://www.streamwithvpn.com/{clean_title}-{int(year)}"

def scrape_movie_data(url, tconst, title, year, endYear, titleType, isAdult, runtime, genres, rating, numVotes):
    """
    Scrape movie/series data from StreamWithVPN
    Returns dictionary with description, cast, and streaming platforms
    """
    try:
        # Add delay to be respectful to the server
        time.sleep(1)
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        # Try the original URL first
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            # print(f"✓ Success with original URL: {url}")
        except requests.exceptions.HTTPError as e:
            if e.response.status_code in [404, 403]:  # Page not found or forbidden
                # Try without year
                url_without_year = generate_streamwithvpn_url(title, None)
                # print(f"⚠ Original URL failed ({e.response.status_code}), trying without year: {url_without_year}")
                
                try:
                    response = requests.get(url_without_year, headers=headers, timeout=10)
                    response.raise_for_status()
                    # print(f"✓ Success with URL without year: {url_without_year}")
                    # Update the URL in the data dictionary for accuracy
                    url = url_without_year
                except requests.exceptions.HTTPError:
                    # print(f"✗ Both URLs failed for {title}")
                    raise  # Re-raise the exception to be caught by outer try-catch
            else:
                raise  # Re-raise non-404/403 errors
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Initialize data dictionary
        movie_data = {
            'tconst': tconst,
            'titleType': titleType,
            'title': title,
            'year': year,
            'endYear': endYear,
            'isAdult': isAdult,
            'runtime': runtime,
            'genres': genres,
            'rating': rating,
            'numVotes': numVotes,
            'description': None,
            'cast': None,
            'streaming_platforms': None,
            'url': url,
            'scrape_status': 'success'
        }
        
        # Extract DESCRIPTION - multiple approaches
        description_element = soup.find('span', class_='rt-Text EntryDetailDescription_contentDescription__tXYGO EntryDetailDescription_expanded__3a0Gs')
        
        if not description_element:
            description_element = soup.find('span', class_=re.compile('EntryDetailDescription_contentDescription'))
        
        if not description_element:
            description_element = soup.find('span', class_=re.compile('contentDescription'))
        
        if not description_element:
            description_element = soup.select_one('span[class*="EntryDetailDescription_contentDescription"]')
        
        if description_element:
            movie_data['description'] = description_element.get_text(strip=True)
            # print(f"✓ Found description for {title}: {movie_data['description'][:100]}...")
        else:
            # print(f"✗ No description found for {title}")
            pass
        """
        # Extract CAST information
        cast_list = []
        # Target container div
        container_div = soup.find('div', class_='rt-Flex rt-r-fd-column rt-r-gap rt-r-px rt-r-pt rt-r-w', style='--gap: 2px; --pl: 16px; --pr: 16px; --pt: 8px; --width: 100%;')
        if container_div:
            cast_spans = container_div.find_all('span', {'data-accent-color': 'gray', 'class_': 'rt-Text rt-r-size-3 rt-r-weight-medium', 'style': 'min-width: 0px; overflow: hidden; text-overflow: ellipsis; white-space: nowrap;'})
            if cast_spans:
                for span in cast_spans:
                    cast_list.append(span.get_text(strip=True))
                movie_data['cast'] = ', '.join(cast_list)
                print(f"✓ Found cast for {title}: {movie_data['cast'][:100]}...")
            else:
                print(f"✗ Span not found for {title}")
        else:
            print(f"✗ Div not found for {title}")

        # Extract STREAMING PLATFORMS information
        platform_elements = soup.find_all('h2', class_='rt-Heading rt-r-size-5 rt-r-weight-medium rt-r-ta-left')
        if platform_elements:
            platforms = [elem.get_text(strip=True) for elem in platform_elements]
            movie_data['streaming_platforms'] = ', '.join(platforms)
        else:
            print(f"✗ No streaming platforms found for {title}")
        """
        
        return movie_data
        
    except requests.RequestException as e:
        print(f"Request error for {title}: {e}")
        return {
            'tconst': tconst,
            'title': title,
            'url': url,
            'description': None,
            'cast': None,
            'streaming_platforms': None,
            'scrape_status': f'request_error: {str(e)}'
        }
    except Exception as e:
        print(f"Parsing error for {title}: {e}")
        return {
            'tconst': tconst,
            'title': title,
            'url': url,
            'description': None,
            'cast': None,
            'streaming_platforms': None,
            'scrape_status': f'parsing_error: {str(e)}'
        }

# Initialize list to store scraped data
scraped_data = []

# Sample scraping for first 2 entries (for faster debugging)
print("Starting web scraping")
sample_df = top_10000_df.head(10) # Change to 10000 for full run (2 for testing)

for index, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="Scraping movies"):
    tconst = row['tconst']
    title = row['primaryTitle']
    year = row['startYear']
    endYear = row['endYear']
    titleType = row['titleType']
    isAdult = row['isAdult']
    runtime = row['runtimeMinutes']
    genres = row['genres']
    rating = row['averageRating']
    numVotes = row['numVotes']
    
    
    # Generate URL
    url = generate_streamwithvpn_url(title, year)
    # print(f"\nScraping: {title} ({year}) - {url}")
    
    # Scrape data
    movie_data = scrape_movie_data(url, tconst, title, year, endYear, titleType, isAdult, runtime, genres, rating, numVotes)
    scraped_data.append(movie_data)

# Convert to DataFrame
scraped_df = pd.DataFrame(scraped_data)

# Display results
print(f"\nScraping completed! Found data for {len(scraped_df)} entries")
print(f"Success rate: {len(scraped_df[scraped_df['scrape_status'] == 'success'])} / {len(scraped_df)}")

# Show sample results
print("\nSample scraped data:")
print(scraped_df[['title', 'url', 'scrape_status', 'description', 'cast']].head())

# Save scraped data
scraped_df.to_csv('data/top10000_final.tsv', sep='\t', index=False)
print("\nSample data saved to 'data/top10000_final.tsv'")



   ---------------------------------------- 0/2 [soupsieve]
   ---------------------------------------- 0/2 [soupsieve]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   -------------------- ------------------- 1/2 [beautifulsoup4]
   ---------------------------------------- 2/2 [beautifulsoup4]



Starting web scraping


Scraping movies: 100%|██████████| 10/10 [00:22<00:00,  2.25s/it]


Scraping completed! Found data for 10 entries
Success rate: 10 / 10

Sample scraped data:
                        title  \
0                Breaking Bad   
1    The Shawshank Redemption   
2  Avatar: The Last Airbender   
3                    The Wire   
4             Game of Thrones   

                                                 url scrape_status  \
0    https://www.streamwithvpn.com/breaking-bad-2008       success   
1  https://www.streamwithvpn.com/the-shawshank-re...       success   
2  https://www.streamwithvpn.com/avatar-the-last-...       success   
3        https://www.streamwithvpn.com/the-wire-2002       success   
4  https://www.streamwithvpn.com/game-of-thrones-...       success   

                                         description  cast  
0  Walter White, a New Mexico chemistry teacher, ...  None  
1  Imprisoned in the 1940s for the double murder ...  None  
2  In a war-torn world of elemental magic, a youn...  None  
3  Told from the points of view of both the B

In [ ]:
# Web scraping script for StreamWithVPN data
%pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup
import time
import re
from urllib.parse import quote
import pandas as pd
from tqdm import tqdm

# Load the top 10,000 weighted ratings data
top_10000_df = pd.read_csv('data/top_10000_weighted_ratings.tsv', sep='\t')

def clean_title_for_url(title):
    """
    Clean and format movie/series title for URL generation
    """
    # Remove special characters and replace spaces with hyphens
    cleaned = re.sub(r'[^\w\s-]', '', title)
    cleaned = re.sub(r'\s+', '-', cleaned.strip())
    return cleaned.lower()

def generate_streamwithvpn_url(title, year):
    """
    Generate StreamWithVPN URL based on title and year
    Example: "The Wolf's Call" (2019) -> "https://www.streamwithvpn.com/the-wolfs-call-2019"
    """
    clean_title = clean_title_for_url(title)
    # Handle cases where year might be NaN or missing
    if pd.isna(year):
        return f"https://www.streamwithvpn.com/{clean_title}"
    else:
        return f"https://www.streamwithvpn.com/{clean_title}-{int(year)}"

def scrape_movie_data(url, tconst, title, year, endYear, titleType, isAdult, runtime, genres, rating, numVotes):
    """
    Scrape movie/series data from StreamWithVPN
    Returns dictionary with description, cast, and streaming platforms
    """
    try:
        # Add delay to be respectful to the server
        time.sleep(1)
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
        except requests.exceptions.HTTPError as e:
            if e.response.status_code in [404, 403]:
                url_without_year = generate_streamwithvpn_url(title, None)
                try:
                    response = requests.get(url_without_year, headers=headers, timeout=10)
                    response.raise_for_status()
                    url = url_without_year
                except requests.exceptions.HTTPError:
                    raise
            else:
                raise
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        movie_data = {
            'tconst': tconst,
            'titleType': titleType,
            'title': title,
            'year': year,
            'endYear': endYear,
            'isAdult': isAdult,
            'runtime': runtime,
            'genres': genres,
            'rating': rating,
            'numVotes': numVotes,
            'description': None,
            'cast': None,
            'streaming_platforms': None,
            'url': url,
            'scrape_status': 'success'
        }
        
        # Extract description
        description_element = soup.find('span', class_='rt-Text EntryDetailDescription_contentDescription__tXYGO EntryDetailDescription_expanded__3a0Gs')
        
        if not description_element:
            description_element = soup.find('span', class_=re.compile('EntryDetailDescription_contentDescription'))
        if not description_element:
            description_element = soup.find('span', class_=re.compile('contentDescription'))
        if not description_element:
            description_element = soup.select_one('span[class*="EntryDetailDescription_contentDescription"]')
        
        if description_element:
            movie_data['description'] = description_element.get_text(strip=True)
            print(f"✓ Found description for {title}: {movie_data['description'][:100]}...")
        else:
            print(f"✗ No description found for {title}")
        
        """
        # Extract CAST information
        cast_list = []
        # Target container div
        container_div = soup.find('div', class_='rt-Flex rt-r-fd-column rt-r-gap rt-r-px rt-r-pt rt-r-w', style='--gap: 2px; --pl: 16px; --pr: 16px; --pt: 8px; --width: 100%;')
        if container_div:
            cast_spans = container_div.find_all('span', {'data-accent-color': 'gray', 'class_': 'rt-Text rt-r-size-3 rt-r-weight-medium', 'style': 'min-width: 0px; overflow: hidden; text-overflow: ellipsis; white-space: nowrap;'})
            if cast_spans:
                for span in cast_spans:
                    cast_list.append(span.get_text(strip=True))
                movie_data['cast'] = ', '.join(cast_list)
                print(f"✓ Found cast for {title}: {movie_data['cast'][:100]}...")
            else:
                print(f"✗ Span not found for {title}")
        else:
            print(f"✗ Div not found for {title}")

        # Extract STREAMING PLATFORMS information
        platform_elements = soup.find_all('h2', class_='rt-Heading rt-r-size-5 rt-r-weight-medium rt-r-ta-left')
        if platform_elements:
            platforms = [elem.get_text(strip=True) for elem in platform_elements]
            movie_data['streaming_platforms'] = ', '.join(platforms)
        else:
            print(f"✗ No streaming platforms found for {title}")
        """
        
        return movie_data
        
    except requests.RequestException as e:
        return {
            'tconst': tconst,
            'title': title,
            'url': url,
            'description': None,
            'cast': None,
            'streaming_platforms': None,
            'scrape_status': f'request_error: {str(e)}'
        }
    except Exception as e:
        return {
            'tconst': tconst,
            'title': title,
            'url': url,
            'description': None,
            'cast': None,
            'streaming_platforms': None,
            'scrape_status': f'parsing_error: {str(e)}'
        }

# Initialize variables
chunk_size = 500  # Process 500 movies at a time
total_chunks = len(top_10000_df) // chunk_size + 1
start_time = time.time()

# Process in chunks
for chunk_start in tqdm(range(0, len(top_10000_df), chunk_size), desc="Processing chunks"):
    chunk_end = min(chunk_start + chunk_size, len(top_10000_df))
    chunk_df = top_10000_df.iloc[chunk_start:chunk_end]
    chunk_data = []
    
    # Process each movie in the chunk
    for _, row in tqdm(chunk_df.iterrows(), total=len(chunk_df), desc=f"Chunk {chunk_start//chunk_size + 1}/{total_chunks}", leave=False):
        try:
            tconst = row['tconst']
            title = row['primaryTitle']
            year = row['startYear']
            endYear = row['endYear']
            titleType = row['titleType']
            isAdult = row['isAdult']
            runtime = row['runtimeMinutes']
            genres = row['genres']
            rating = row['averageRating']
            numVotes = row['numVotes']
            
            url = generate_streamwithvpn_url(title, year)
            movie_data = scrape_movie_data(url, tconst, title, year, endYear, 
                                         titleType, isAdult, runtime, genres, 
                                         rating, numVotes)
            chunk_data.append(movie_data)
            
        except Exception as e:
            print(f"\nError processing {title}: {str(e)}")
            continue
    
    # Save chunk progress
    chunk_df = pd.DataFrame(chunk_data)
    chunk_df.to_csv(f'data/temp_scrape_chunk_{chunk_start}.csv', sep='\t', index=False)
    
    # Print progress stats
    elapsed = time.time() - start_time
    processed = chunk_end
    remaining = len(top_10000_df) - processed
    rate = processed / elapsed
    eta = remaining / rate if rate > 0 else 0
    
    print(f"\nChunk {chunk_start}-{chunk_end} completed")
    print(f"Processed {processed:,}/{len(top_10000_df):,} movies in {elapsed/3600:.2f} hours")
    print(f"Estimated time remaining: {eta/3600:.2f} hours")
    print(f"Success rate in chunk: {len(chunk_df[chunk_df['scrape_status'] == 'success'])} / {len(chunk_df)}")

# Combine all chunks
print("\nCombining chunks...")
all_chunks = []
for chunk_start in range(0, len(top_10000_df), chunk_size):
    try:
        chunk = pd.read_csv(f'data/temp_scrape_chunk_{chunk_start}.csv', sep='\t')
        all_chunks.append(chunk)
    except Exception as e:
        print(f"Error reading chunk {chunk_start}: {str(e)}")

scraped_df = pd.concat(all_chunks, ignore_index=True)

# Save final results
scraped_df.to_csv('data/top10000_final.tsv', sep='\t', index=False)

# Print final statistics
total_time = time.time() - start_time
print(f"\nProcessing completed in {total_time/3600:.2f} hours")
print(f"Total movies processed: {len(scraped_df):,}")
print(f"Overall success rate: {len(scraped_df[scraped_df['scrape_status'] == 'success']):,} / {len(scraped_df):,}")

# Clean up temporary files
import os
for chunk_start in range(0, len(top_10000_df), chunk_size):
    try:
        os.remove(f'data/temp_scrape_chunk_{chunk_start}.csv')
    except:
        pass

Note: you may need to restart the kernel to use updated packages.


Processing chunks:   5%|▌         | 1/20 [17:18<5:28:53, 1038.60s/it]


Chunk 0-500 completed
Processed 500/10,000 movies in 0.29 hours
Estimated time remaining: 5.48 hours
Success rate in chunk: 447 / 500


Processing chunks:  10%|█         | 2/20 [34:34<5:11:01, 1036.76s/it]


Chunk 500-1000 completed
Processed 1,000/10,000 movies in 0.58 hours
Estimated time remaining: 5.19 hours
Success rate in chunk: 448 / 500


...


Chunk 8500-9000 completed
Processed 9,000/10,000 movies in 5.06 hours
Estimated time remaining: 0.56 hours
Success rate in chunk: 346 / 500


Processing chunks:  95%|█████████▌| 19/20 [5:19:53<16:26, 986.78s/it]


Chunk 9000-9500 completed
Processed 9,500/10,000 movies in 5.33 hours
Estimated time remaining: 0.28 hours
Success rate in chunk: 352 / 500


Processing chunks: 100%|██████████| 20/20 [5:36:08<00:00, 1008.43s/it]



Chunk 9500-10000 completed
Processed 10,000/10,000 movies in 5.60 hours
Estimated time remaining: 0.00 hours
Success rate in chunk: 352 / 500

Combining chunks...

Processing completed in 5.60 hours
Total movies processed: 10,000
Overall success rate: 7,988 / 10,000


In [4]:
# Load the previously scraped data
scraped_df = pd.read_csv('data/top10000_final.tsv', sep='\t')

# Identify failed scraping attempts
failed_scrapes = scraped_df[scraped_df['scrape_status'] != 'success']
print(f"\nFound {len(failed_scrapes)} failed scraping attempts")

# Initialize variables for retry
chunk_size = 500
total_chunks = len(failed_scrapes) // chunk_size + 1
start_time = time.time()
retry_data = []

# Process failed scrapes in chunks
for chunk_start in tqdm(range(0, len(failed_scrapes), chunk_size), desc="Retrying failed scrapes"):
    chunk_end = min(chunk_start + chunk_size, len(failed_scrapes))
    chunk_df = failed_scrapes.iloc[chunk_start:chunk_end]
    chunk_data = []
    
    # Process each failed movie in the chunk
    for _, row in tqdm(chunk_df.iterrows(), total=len(chunk_df), desc=f"Retry Chunk {chunk_start//chunk_size + 1}/{total_chunks}", leave=False):
        try:
            tconst = row['tconst']
            title = row['title']
            year = row['year']
            endYear = row['endYear']
            titleType = row['titleType']
            isAdult = row['isAdult']
            runtime = row['runtime']
            genres = row['genres']
            rating = row['rating']
            numVotes = row['numVotes']
            
            # Try scraping again with increased delay
            time.sleep(2)  # Increased delay for retry attempts
            url = generate_streamwithvpn_url(title, year)
            movie_data = scrape_movie_data(url, tconst, title, year, endYear, 
                                         titleType, isAdult, runtime, genres, 
                                         rating, numVotes)
            chunk_data.append(movie_data)
            
        except Exception as e:
            print(f"\nError processing {title}: {str(e)}")
            continue
    
    # Save retry progress
    retry_chunk_df = pd.DataFrame(chunk_data)
    retry_data.extend(chunk_data)
    
    # Print progress stats
    elapsed = time.time() - start_time
    processed = chunk_end
    remaining = len(failed_scrapes) - processed
    rate = processed / elapsed
    eta = remaining / rate if rate > 0 else 0
    
    print(f"\nRetry Chunk {chunk_start}-{chunk_end} completed")
    print(f"Processed {processed:,}/{len(failed_scrapes):,} movies in {elapsed/3600:.2f} hours")
    print(f"Estimated time remaining: {eta/3600:.2f} hours")
    print(f"Success rate in retry chunk: {len(retry_chunk_df[retry_chunk_df['scrape_status'] == 'success'])} / {len(retry_chunk_df)}")

# Convert retry results to DataFrame
retry_df = pd.DataFrame(retry_data)

# Update original DataFrame with successful retries
successful_retries = retry_df[retry_df['scrape_status'] == 'success']
for _, retry_row in successful_retries.iterrows():
    scraped_df.loc[scraped_df['tconst'] == retry_row['tconst']] = retry_row

# Save updated results
scraped_df.to_csv('data/top10000_final_with_retries.tsv', sep='\t', index=False)

# Print final statistics
total_time = time.time() - start_time
print(f"\nRetry processing completed in {total_time/3600:.2f} hours")
print(f"Initial failed scrapes: {len(failed_scrapes):,}")
print(f"Successful retries: {len(successful_retries):,}")
print(f"Final success rate: {len(scraped_df[scraped_df['scrape_status'] == 'success']):,} / {len(scraped_df):,}")


Found 2012 failed scraping attempts


Retrying failed scrapes:  20%|██        | 1/5 [31:14<2:04:57, 1874.31s/it]


Retry Chunk 0-500 completed
Processed 500/2,012 movies in 0.52 hours
Estimated time remaining: 1.57 hours
Success rate in retry chunk: 0 / 500


Retrying failed scrapes:  40%|████      | 2/5 [1:02:29<1:33:43, 1874.54s/it]


Retry Chunk 500-1000 completed
Processed 1,000/2,012 movies in 1.04 hours
Estimated time remaining: 1.05 hours
Success rate in retry chunk: 0 / 500


...


Retry Chunk 1000-1500 completed
Processed 1,500/2,012 movies in 1.56 hours
Estimated time remaining: 0.53 hours
Success rate in retry chunk: 0 / 500


Retrying failed scrapes:  80%|████████  | 4/5 [2:05:03<31:16, 1876.44s/it]  


Retry Chunk 1500-2000 completed
Processed 2,000/2,012 movies in 2.08 hours
Estimated time remaining: 0.01 hours
Success rate in retry chunk: 0 / 500


Retrying failed scrapes: 100%|██████████| 5/5 [2:05:49<00:00, 1509.84s/it]


Retry Chunk 2000-2012 completed
Processed 2,012/2,012 movies in 2.10 hours
Estimated time remaining: 0.00 hours
Success rate in retry chunk: 0 / 12

Retry processing completed in 2.10 hours
Initial failed scrapes: 2,012
Successful retries: 0
Final success rate: 7,988 / 10,000


## Unify Description and Cast data in 1 file

In [6]:
import numpy as np
import pandas as pd

top10000_cast = pd.read_csv('data/cast_data/top_10000_with_cast.tsv', sep='\t')
top10000_description = pd.read_csv('data/top10000_final.tsv', sep='\t')

# Merge cast and reviews data on the 'tconst' column by getting only the top10000_cast columns and adding the description from top10000_description
merged_df = top10000_cast.merge(top10000_description[['tconst', 'description']], on='tconst', how='inner')
# Save the merged dataframe to a new TSV file
merged_df.to_csv('data/top_10000_with_cast_and_description.tsv', sep='\t', index=False)



In [ ]:
top10000 = pd.read_csv('data/top_10000_with_cast_and_description.tsv', sep='\t')

# Try to read the reviews file with error handling
try:
    reviews = pd.read_csv('data/reviews_data/filtered_reviews2.csv', sep='\t')
    print("Reviews loaded successfully")
except pd.errors.ParserError as e:
    print(f"Parser error: {e}")
    # Try with different options to handle malformed data
    try:
        reviews = pd.read_csv('data/reviews_data/filtered_reviews2.csv', 
                            sep='\t', 
                            on_bad_lines='skip',  # Skip bad lines
                            engine='python')     # Use Python engine for better error handling
        print("Reviews loaded with some lines skipped due to parsing errors")
    except Exception as e2:
        print(f"Still failed: {e2}")
        # Try with comma separator instead
        try:
            reviews = pd.read_csv('data/reviews_data/filtered_reviews2.csv', 
                                sep=',',  # Try comma separator
                                on_bad_lines='skip')
            print("Reviews loaded using comma separator")
        except Exception as e3:
            print(f"All attempts failed: {e3}")
            reviews = None

# Get different values of isAdult
#print(top10000['isAdult'].value_counts())
# Remove isAdult column
#top10000 = top10000.drop(columns=['isAdult'])
#top10000.to_csv('data/top_10000_with_cast_and_description.tsv', sep='\t', index=False)

#print((top10000[top10000['primaryTitle'] != top10000['originalTitle']]).shape[0])

# Total null descriptions
total_nulls = top10000[top10000['description'].isna()].shape[0]
print(f"Total null descriptions: {total_nulls}")

# Analyze null descriptions in chunks of 500
chunk_size = 500
total_entries = len(top10000)

print("\nNull descriptions by 500-entry chunks:")
print("-" * 40)

for chunk_start in range(0, total_entries, chunk_size):
    chunk_end = min(chunk_start + chunk_size, total_entries)
    chunk_data = top10000.iloc[chunk_start:chunk_end]
    
    null_count = chunk_data[chunk_data['description'].isna()].shape[0]
    chunk_total = len(chunk_data)
    percentage = (null_count / chunk_total) * 100 if chunk_total > 0 else 0
    
    print(f"Entries {chunk_start+1:4d}-{chunk_end:4d}: {null_count:3d} nulls out of {chunk_total:3d} ({percentage:5.1f}%)")

print("-" * 40)
print(f"Overall: {total_nulls} nulls out of {total_entries} ({(total_nulls/total_entries)*100:.1f}%)")

'''
# Only show reviews if they loaded successfully
if reviews is not None:
    print(f"\nReviews data shape: {reviews.shape}")
    print(reviews[reviews['tconst']=='tt0903747'])
else:
    print("\nCould not load reviews data")

# Remove lines with no description
top10000_cleaned = top10000.dropna(subset=['description'])
print(f"\nAfter removing null descriptions, new shape: {top10000_cleaned.shape}")
top10000_cleaned.to_csv('data/FinalData.tsv', sep='\t', index=False)
'''

Parser error: Error tokenizing data. C error: Expected 1 fields in line 5967, saw 2

Reviews loaded with some lines skipped due to parsing errors
Total null descriptions: 2024

Null descriptions by 500-entry chunks:
----------------------------------------
Entries    1- 500:  54 nulls out of 500 ( 10.8%)
Entries  501-1000:  52 nulls out of 500 ( 10.4%)
Entries 1001-1500:  62 nulls out of 500 ( 12.4%)
Entries 1501-2000:  75 nulls out of 500 ( 15.0%)
Entries 2001-2500:  51 nulls out of 500 ( 10.2%)
Entries 2501-3000:  88 nulls out of 500 ( 17.6%)
Entries 3001-3500:  82 nulls out of 500 ( 16.4%)
Entries 3501-4000:  90 nulls out of 500 ( 18.0%)
Entries 4001-4500:  92 nulls out of 500 ( 18.4%)
Entries 4501-5000:  94 nulls out of 500 ( 18.8%)
Entries 5001-5500: 108 nulls out of 500 ( 21.6%)
Entries 5501-6000: 111 nulls out of 500 ( 22.2%)
Entries 6001-6500: 113 nulls out of 500 ( 22.6%)
Entries 6501-7000: 122 nulls out of 500 ( 24.4%)
Entries 7001-7500: 122 nulls out of 500 ( 24.4%)
Entries 